In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Atoms
from ase.io import read

# ML
from regression import SparseKRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from split import simple_split

# Utilities
from selection import FPS, random_selection
import multiprocessing
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist
import itertools

# SOAP
from soap import quippy_soap, librascal_soap

# Initial setup

In [3]:
# Setup SOAP parameters
soap_hyperparameters = dict(max_radial=12,
                            max_angular=9,
                            interaction_cutoff=6.0,
                            cutoff_smooth_width=0.3,
                            gaussian_sigma_constant=0.3)

# Load DEEM_10k

In [4]:
# Load DEEM 10k
deem_10k = read('../Raw_Data/DEEM_10k/DEEM_10000.xyz', index=':')

In [5]:
# Extract energies per Si atom
deem_10k_volumes = []
deem_10k_energies = []
n_Si = []

for structure in deem_10k:
    Z = structure.get_atomic_numbers()
    n_Si.append(np.count_nonzero(Z == 14))
    deem_10k_volumes.append(structure.cell.volume / n_Si[-1])
    deem_10k_energies.append(structure.info['Energy_per_Si'])
    
deem_10k_volumes = np.asarray(deem_10k_volumes)
deem_10k_energies = np.asarray(deem_10k_energies)

# Check DEEM database energies vs. GULP

In [6]:
# Load GULP energies, which are in the same order as the loaded structures
deem_10k_energies_gulp = np.loadtxt('../Raw_Data/GULP/DEEM_10k/Energies_DEEM.dat', usecols=8)

# Check that GULP energies are similar to the database energies
abs_err = np.abs(deem_10k_energies - deem_10k_energies_gulp)

# mean, median, and maximum absolute error
print(np.mean(abs_err))
print(np.median(abs_err))
print(np.amax(abs_err))

0.08162864922236804
0.07408146875241073
8.360833570570321


# Truncate DEEM dataset for testing

In [7]:
n = 2000
stride = len(deem_10k) // n
structures = deem_10k[::stride]
structure_volumes = deem_10k_volumes[::stride]
structure_energies = deem_10k_energies[::stride]
print(len(structures))

2000


In [8]:
f = 0.75
idxs = np.arange(0, len(structures))
np.random.shuffle(idxs)
train_idxs = idxs[0:int(n*f)]
test_idxs = idxs[int(n*f):]

# Test FPS on environment SOAPs vs. FPS on mean SOAPs

In [9]:
# Compute environment SOAPs
soaps = librascal_soap(structures, [14],
                         **soap_hyperparameters)

In [10]:
# Compute avg SOAPs
soaps_avg = librascal_soap(structures, [14],
                              **soap_hyperparameters,
                              average=True)
soaps_avg = np.asarray(soaps_avg)

In [11]:
n_components = 500

In [12]:
# FPS on environment SOAPs
fps, _ = FPS(np.concatenate([soaps[i] for i in train_idxs], axis=0).T, n=n_components, start=0)
print(fps.size)

500


In [13]:
# FPS on average SOAPs
fps_avg, _ = FPS(soaps_avg[train_idxs, :].T, n=n_components, start=0)
print(fps_avg.size)

500


In [14]:
print(len(np.setdiff1d(fps, fps_avg)))
print(len(np.setdiff1d(fps_avg, fps)))

70
70


# Truncate the SOAPs

In [15]:
soaps_avg = [soap[:, fps_avg] for soap in soaps]
soaps = [soap[:, fps] for soap in soaps]

In [16]:
print(np.concatenate(soaps, axis=0).shape)
print(np.concatenate(soaps_avg, axis=0).shape)

(100527, 500)
(100527, 500)


# Get representative environments

In [17]:
n_representatives = 2000
representatives, _ = FPS(np.concatenate([soaps[i] for i in train_idxs], axis=0), n=n_representatives)
representatives_avg, _ = FPS(np.concatenate([soaps_avg[i] for i in train_idxs], axis=0), n=n_representatives)

In [18]:
soaps_rep = np.concatenate([soaps[i] for i in train_idxs], axis=0)[representatives, :]
soaps_rep_avg = np.concatenate([soaps_avg[i] for i in train_idxs], axis=0)[representatives_avg, :]

In [19]:
print(soaps_rep.shape)
print(soaps_rep_avg.shape)

(2000, 500)
(2000, 500)


# Center properties and build kernels

In [20]:
# Center properties
avg_volume = np.mean(structure_volumes[train_idxs])
avg_energy = np.mean(structure_energies[train_idxs])

structure_volumes -= avg_volume
structure_energies -= avg_energy

In [21]:
# Build sparse linear kernels
zeta = 1
KMM_linear = build_kernel(soaps_rep, soaps_rep,
                         kernel='linear', zeta=zeta)
KNM_train_linear = build_kernel([soaps[i] for i in train_idxs], soaps_rep, 
                                kernel='linear', zeta=zeta)
KNM_test_linear = build_kernel([soaps[i] for i in test_idxs], soaps_rep, 
                               kernel='linear', zeta=zeta)

KMM_linear_avg = build_kernel(soaps_rep_avg, soaps_rep_avg,
                             kernel='linear', zeta=zeta)
KNM_train_linear_avg = build_kernel([soaps_avg[i] for i in train_idxs], soaps_rep_avg, 
                                    kernel='linear', zeta=zeta)
KNM_test_linear_avg = build_kernel([soaps_avg[i] for i in test_idxs], soaps_rep_avg, 
                                   kernel='linear', zeta=zeta)

In [22]:
# Build sparse Gaussian kernels
gamma = 0.5
KMM_gaussian = build_kernel(soaps_rep, soaps_rep,
                           kernel='gaussian', gamma=gamma)
KNM_train_gaussian = build_kernel([soaps[i] for i in train_idxs], soaps_rep, 
                                  kernel='gaussian', gamma=gamma)
KNM_test_gaussian = build_kernel([soaps[i] for i in test_idxs], soaps_rep, 
                                 kernel='gaussian', gamma=gamma)

KMM_gaussian_avg = build_kernel(soaps_rep_avg, soaps_rep_avg,
                               kernel='gaussian', gamma=gamma)
KNM_train_gaussian_avg = build_kernel([soaps_avg[i] for i in train_idxs], soaps_rep_avg, 
                                      kernel='gaussian', gamma=gamma)
KNM_test_gaussian_avg = build_kernel([soaps_avg[i] for i in test_idxs], soaps_rep_avg, 
                                     kernel='gaussian', gamma=gamma)

# Kernel speed

In [42]:
K = build_kernel(soaps_rep, soaps_rep, kernel='gaussian', gamma=1.0)

In [43]:
%%timeit
build_kernel(soaps_rep, soaps_rep, kernel='gaussian', gamma=3.0)

991 ms ± 860 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
K**3.0

172 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
print(build_kernel(soaps_rep, soaps_rep, kernel='gaussian', gamma=3.0))
print(K**3.0)

[[1.         0.33513696 0.38982442 ... 0.69105943 0.81470392 0.83777551]
 [0.33513696 1.         0.25213475 ... 0.39278405 0.33036088 0.48896545]
 [0.38982442 0.25213475 1.         ... 0.7012866  0.54833497 0.4063832 ]
 ...
 [0.69105943 0.39278405 0.7012866  ... 1.         0.83186931 0.71877921]
 [0.81470392 0.33036088 0.54833497 ... 0.83186931 1.         0.78297762]
 [0.83777551 0.48896545 0.4063832  ... 0.71877921 0.78297762 1.        ]]
[[1.         0.33513696 0.38982442 ... 0.69105943 0.81470392 0.83777551]
 [0.33513696 1.         0.25213475 ... 0.39278405 0.33036088 0.48896545]
 [0.38982442 0.25213475 1.         ... 0.7012866  0.54833497 0.4063832 ]
 ...
 [0.69105943 0.39278405 0.7012866  ... 1.         0.83186931 0.71877921]
 [0.81470392 0.33036088 0.54833497 ... 0.83186931 1.         0.78297762]
 [0.83777551 0.48896545 0.4063832  ... 0.71877921 0.78297762 1.        ]]


# Volume regression

## Linear

In [46]:
# SOAPs
sigma = 0.1**2
delta = np.var(structure_volumes[train_idxs])*KMM_linear.shape[0]/np.trace(KMM_linear)

skrr = SparseKRR(sigma=sigma, reg=1.0E-1)
skrr.fit(KNM_train_linear*delta, KMM_linear*delta, structure_volumes[train_idxs]*delta)
predicted_volumes = skrr.transform(KNM_test_linear)
print(np.mean(np.abs(structure_volumes[test_idxs] - predicted_volumes)))

1.1394364632306542


In [47]:
# Avg SOAPs with avg FPS
sigma = 0.1**2
delta = np.var(structure_volumes[train_idxs])*KMM_linear_avg.shape[0]/np.trace(KMM_linear_avg)

skrr_avg = SparseKRR(sigma=sigma, reg=1.0E-1)
skrr_avg.fit(KNM_train_linear_avg*delta, KMM_linear_avg*delta, structure_volumes[train_idxs]*delta)
predicted_volumes_avg = skrr_avg.transform(KNM_test_linear_avg)
print(np.mean(np.abs(structure_volumes[test_idxs] - predicted_volumes_avg)))

1.133300566550664


## Gaussian

In [48]:
# SOAPs
sigma = 0.1**2
delta = np.var(structure_volumes[train_idxs])*KMM_gaussian.shape[0]/np.trace(KMM_gaussian)

skrr = SparseKRR(sigma=sigma, reg=1.0E-1)
skrr.fit(KNM_train_gaussian*delta, KMM_gaussian*delta, structure_volumes[train_idxs]*delta)
predicted_volumes = skrr.transform(KNM_test_gaussian)
print(np.mean(np.abs(structure_volumes[test_idxs] - predicted_volumes)))

1.1304934982923796


In [49]:
# Avg SOAPs with avg FPS
sigma = 0.1**2
delta = np.var(structure_volumes[train_idxs])*KMM_gaussian_avg.shape[0]/np.trace(KMM_gaussian_avg)

skrr_avg = SparseKRR(sigma=sigma, reg=1.0E-1)
skrr_avg.fit(KNM_train_gaussian_avg*delta, KMM_gaussian_avg*delta, structure_volumes[train_idxs]*delta)
predicted_volumes_avg = skrr_avg.transform(KNM_test_gaussian_avg)
print(np.mean(np.abs(structure_volumes[test_idxs] - predicted_volumes_avg)))

1.119316543995585


# Energy regression

## Linear

In [50]:
# SOAPs
sigma = 0.01**2
delta = np.var(structure_energies[train_idxs])*KMM_linear.shape[0]/np.trace(KMM_linear)

skrr = SparseKRR(sigma=sigma, reg=1.0E-3)
skrr.fit(KNM_train_linear*delta, KMM_linear*delta, structure_energies[train_idxs]*delta)
predicted_energies = skrr.transform(KNM_test_linear)
print(np.mean(np.abs(structure_energies[test_idxs] - predicted_energies)))

0.6400137389996662


In [51]:
# Avg SOAPs with avg FPS
sigma = 0.01**2
delta = np.var(structure_energies[train_idxs])*KMM_linear_avg.shape[0]/np.trace(KMM_linear_avg)

skrr_avg = SparseKRR(sigma=sigma, reg=1.0E-3)
skrr_avg.fit(KNM_train_linear_avg*delta, KMM_linear_avg*delta, structure_energies[train_idxs]*delta)
predicted_energies_avg = skrr_avg.transform(KNM_test_linear_avg)
print(np.mean(np.abs(structure_energies[test_idxs] - predicted_energies_avg)))

0.6283474537982984


## Gaussian

In [52]:
# SOAPs
sigma = 0.01**2
delta = np.var(structure_energies[train_idxs])*KMM_gaussian.shape[0]/np.trace(KMM_gaussian)

skrr = SparseKRR(sigma=sigma, reg=1.0E-3)
skrr.fit(KNM_train_gaussian*delta, KMM_gaussian*delta, structure_energies[train_idxs]*delta)
predicted_energies = skrr.transform(KNM_test_gaussian)
print(np.mean(np.abs(structure_energies[test_idxs] - predicted_energies)))

0.662661060440006


In [53]:
# Avg SOAPs with avg FPS
sigma = 0.01**2
delta = np.var(structure_energies[train_idxs])*KMM_gaussian_avg.shape[0]/np.trace(KMM_gaussian_avg)

skrr_avg = SparseKRR(sigma=sigma, reg=1.0E-3)
skrr_avg.fit(KNM_train_gaussian_avg*delta, KMM_gaussian_avg*delta, structure_energies[train_idxs]*delta)
predicted_energies_avg = skrr_avg.transform(KNM_test_gaussian_avg)
print(np.mean(np.abs(structure_energies[test_idxs] - predicted_energies_avg)))

0.6627608443840436


# Distance computation

In [19]:
# Build some large matrices
XA = np.random.random_sample(size=(10000, 500))
XB = np.random.random_sample(size=(10000, 500))

## cdist

In [20]:
%%timeit
D = cdist(XA, XB, metric='sqeuclidean')

28.6 s ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
D = cdist(XA, XB, metric='euclidean')**2

28.9 s ± 51.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
D = cdist(XA, XB, metric='sqeuclidean')
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


In [23]:
D = cdist(XA, XB, metric='euclidean')**2
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


## pairwise_distances

In [24]:
%%timeit
D = pairwise_distances(XA, XB, metric='sqeuclidean', n_jobs=-1)

15.6 s ± 64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
D = pairwise_distances(XA, XB, metric='sqeuclidean')

28.7 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
D = pairwise_distances(XA, XB, metric='euclidean', n_jobs=-1)**2

1.74 s ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
D = pairwise_distances(XA, XB, metric='euclidean')**2

1.62 s ± 8.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
D = pairwise_distances(XA, XB, metric='sqeuclidean', n_jobs=-1)
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


In [29]:
D = pairwise_distances(XA, XB, metric='sqeuclidean')
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


In [30]:
D = pairwise_distances(XA, XB, metric='euclidean', n_jobs=-1)**2
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


In [31]:
D = pairwise_distances(XA, XB, metric='euclidean')**2
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


## Multiprocessing (unusably slow)

## Faster squared euclidean distance

In [32]:
def sqeuclidean_distance(XA, XB):
    XA2 = np.sum(XA**2, axis=1).reshape((-1, 1))
    XB2 = np.sum(XB**2, axis=1).reshape((1, -1))
    D = XA2 + XB2 - 2*np.matmul(XA, XB.T)
    return D

In [33]:
%%timeit
D = sqeuclidean_distance(XA, XB)

1.37 s ± 47.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
D = sqeuclidean_distance(XA, XB)
print(D)

[[85.62247385 84.28213009 75.5079755  ... 78.35619996 79.37481873
  89.13174948]
 [86.40241542 88.90474247 80.26014864 ... 86.40137103 84.52379281
  84.71373093]
 [83.50665963 84.30849822 84.86033297 ... 86.35759096 85.86527811
  81.10208381]
 ...
 [83.93677987 83.98809912 75.76069389 ... 82.52475046 83.5778463
  77.23209771]
 [82.47088021 76.94440533 74.61938182 ... 81.28199375 73.46484975
  81.3682162 ]
 [84.25210381 87.54029826 85.43774998 ... 91.05228089 87.87675652
  92.1036993 ]]


# Kernel Computation

## Multiprocessing

In [36]:
# Gaussian kernel with cdist
def g_kernel_cdist(XA, XB, gamma=1.0, row_mean=True, col_mean=True):
    K = cdist(XA, XB, metric='sqeuclidean')
    K = np.exp(-gamma*K)
    
    if row_mean and col_mean:
        K = np.mean(K)
    elif row_mean:
        K = np.mean(K, axis=0)
    elif col_mean:
        K = np.mean(K, axis=1)
        
    return K

# Gaussian kernel pairwise_distances
def g_kernel_pairwise(XA, XB, gamma=1.0, row_mean=True, col_mean=True):
    K = pairwise_distances(XA, XB, metric='euclidean')**2
    K = np.exp(-gamma*K)
    
    if row_mean and col_mean:
        K = np.mean(K)
    elif row_mean:
        K = np.mean(K, axis=0)
    elif col_mean:
        K = np.mean(K, axis=1)
        
    return K

In [37]:
# Build some large matrices
XA = np.random.random_sample(size=(10000, 500))
XB = np.random.random_sample(size=(10000, 500))

XA /= 500
XB /= 500

In [38]:
split_A = np.random.randint(0, 10000, 100)
split_B = np.random.randint(0, 10000, 100)

split_A.sort()
split_B.sort()

In [39]:
XA = np.split(XA, split_A)
XB = np.split(XB, split_B)

XA = [xa for xa in XA if xa.size > 0]
XB = [xb for xb in XB if xb.size > 0]

In [40]:
%%timeit
with multiprocessing.Pool() as pool:
    out = pool.starmap(g_kernel_cdist, itertools.product(XA, XB))

np.reshape(out, (len(XA), len(XB)))

5.24 s ± 58.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
with multiprocessing.Pool() as pool:
    out = pool.starmap(g_kernel_pairwise, itertools.product(XA, XB))
    
np.reshape(out, (len(XA), len(XB)))

10.5 s ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
build_kernel(XA, XB, kernel='gaussian', gamma=1.0) # 8s with sklearn

6.33 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
with multiprocessing.Pool() as pool:
    out = pool.starmap(g_kernel_cdist, itertools.product(XA, XB))

out = np.reshape(out, (len(XA), len(XB)))
print(out)

[[0.99966626 0.99966677 0.99966923 ... 0.99966802 0.99966753 0.99966827]
 [0.99966628 0.99966753 0.99966821 ... 0.99966728 0.99966588 0.99966571]
 [0.99966574 0.9996671  0.99966761 ... 0.99966675 0.99966523 0.99966493]
 ...
 [0.99966609 0.99966749 0.99966794 ... 0.99966693 0.99966558 0.99966536]
 [0.99966645 0.99966687 0.99966817 ... 0.99966707 0.99966594 0.99966516]
 [0.99966538 0.99966662 0.99966719 ... 0.99966644 0.99966492 0.99966436]]


In [44]:
with multiprocessing.Pool() as pool:
    out = pool.starmap(g_kernel_pairwise, itertools.product(XA, XB))

out = np.reshape(out, (len(XA), len(XB)))
print(out)

[[0.99966626 0.99966677 0.99966923 ... 0.99966802 0.99966753 0.99966827]
 [0.99966628 0.99966753 0.99966821 ... 0.99966728 0.99966588 0.99966571]
 [0.99966574 0.9996671  0.99966761 ... 0.99966675 0.99966523 0.99966493]
 ...
 [0.99966609 0.99966749 0.99966794 ... 0.99966693 0.99966558 0.99966536]
 [0.99966645 0.99966687 0.99966817 ... 0.99966707 0.99966594 0.99966516]
 [0.99966538 0.99966662 0.99966719 ... 0.99966644 0.99966492 0.99966436]]


In [45]:
out = build_kernel(XA, XB, kernel='gaussian', gamma=1.0)
print(out)

[[0.99966626 0.99966677 0.99966923 ... 0.99966802 0.99966753 0.99966827]
 [0.99966628 0.99966753 0.99966821 ... 0.99966728 0.99966588 0.99966571]
 [0.99966574 0.9996671  0.99966761 ... 0.99966675 0.99966523 0.99966493]
 ...
 [0.99966609 0.99966749 0.99966794 ... 0.99966693 0.99966558 0.99966536]
 [0.99966645 0.99966687 0.99966817 ... 0.99966707 0.99966594 0.99966516]
 [0.99966538 0.99966662 0.99966719 ... 0.99966644 0.99966492 0.99966436]]
